In [ ]:
#HW 11 Problem 3 and 4 (I explained functions through #)

In [128]:
import urllib.request
response = urllib.request.urlretrieve('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', "irisdata")

In [129]:
N=5
with open("irisdata") as myfile:
    head = myfile.readlines(N)
    print (head)

['5.1,3.5,1.4,0.2,Iris-setosa\n']


In [130]:
# Softmax example in TF using the classical Iris dataset
# Download iris.data from https://archive.ics.uci.edu/ml/datasets/Iris

import tensorflow as tf
import os
#Define parameters for the model
batch_size = 128

# weights form a matrix, not a column vector, one "weight vector" per class.
W = tf.Variable(tf.zeros([4, 3]), name="weights")
# so do the biases, one per class.
b = tf.Variable(tf.zeros([3], name="bias"))

X = tf.placeholder(tf.float32) 
Y = tf.placeholder(tf.int32)

In [131]:
# line 13: define a function X(W^T)+b. This is a product of 2 vectors and a bias.
#Combine_inputs is a named argument, comebine inputs into a single output
def combine_inputs(X): 
    return tf.matmul(X, W) + b

In [132]:
# line 17: compute inference model over data X and return result. Tensorflow contains an embedded activation of the
#softmax function. 
def inference(X): 
    return tf.nn.softmax(combine_inputs(X))

#def hw11p3(Y):
    #return Y
# loss function tells us how good our prediction is, in this case, we use softmax cross entropy function.
# This version of softmax function lets you work with training sets containing the probabilities
# of each example to belong to every class. The final output of the model will always be one
# single class value.
# X is the actual value, Y is the predicting value
logits = tf.matmul(X, W) + b 
softmax = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,labels=Y, name='loss')
loss = tf.reduce_mean(softmax)

In [133]:
# line 25: read in the file irisdata
# Queues the filneame for an input, then reads the text in file one line at a time, 
# then puts them in a "key, value" format stings
def read_csv(batch_size, irisdata, record_defaults):
    filename_queue = tf.train.string_input_producer([os.path.dirname('Users/Devon/Downloads') + irisdata])

    reader = tf.TextLineReader(skip_header_lines=1)
    key, value = reader.read(filename_queue)

    # line 34
    # decode_csv will convert a Tensor from type string (the text line) in
    # a tuple of tensor columns with the specified defaults, which also
    # sets the data type for each column   
    decoded = tf.decode_csv(value, record_defaults=record_defaults)

    # line 37 
    # creates batches by randomly shuffling tensors
    # batch actually reads the file and loads "batch_size" rows in a single tensor
    return tf.train.shuffle_batch(decoded,
                                  batch_size=batch_size,
                                  capacity=batch_size * 50,
                                  min_after_dequeue=batch_size)

In [134]:
# line 43 : Read/generate input training data X and expected outputs Y
def inputs():
    # separates data into labelled columns for training
    sepal_length, sepal_width, petal_length, petal_width, label =\
        read_csv(100, "irisdata", [[0.0], [0.0], [0.0], [0.0], [""]])
    # line 49    
    # convert class names to a 0 based class index.
    # label_number tells what number is of that label, individually numbering labels
    # creates an index of the different Iris flower types to be identified
    # I changed tf.pack to tf.stack
    label_number = tf.to_int32(tf.argmax(tf.to_int32(tf.stack([
        tf.equal(label, ["Iris-setosa"]),
        tf.equal(label, ["Iris-versicolor"]),
        tf.equal(label, ["Iris-virginica"])
    ])), 0))

    # line 57
    # Based on label_number, we pack all the features that we care about in a single matrix;
    # We then transpose to have a matrix with one example per row and one feature per column.
    features = tf.transpose(tf.stack([sepal_length, sepal_width, petal_length, petal_width]))

    return features, label_number

In [135]:
# We perform gradient descent, loss function was calculated. Its looking for w1, w2, w3, etc.. 
# given the model combine_inputs = (X(W^T))+b
def train(loss):
    learning_rate = 0.01
    return tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

In [136]:
# line 67  :Evaluate the resulting trained model
def evaluate(sess, X, Y):
#line 69
#cast a tensor to a new type in this case int32. Argmax returns the index with the largest value across axes of a tensor.
# doing all of this to get a predicted value
    predicted = tf.cast(tf.arg_max(inference(X), 1), tf.int32)

 # line 71
 #checking to see if the prediction is accurate and outputting the result
    print (sess.run(tf.reduce_mean(tf.cast(tf.equal(predicted, Y), tf.float32))))    

In [138]:
# Launch the graph in a session, setup boilerplate
with tf.Session() as sess:

    #tf.initialize_all_variables().run()
    tf.global_variables_initializer
    
    X, Y = inputs()

    train_op = train(loss)

    coord = tf.train.Coordinator()
    
    #line 85
    # starts all queue runners collected in the graph. It returns the list of all threads.
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    # actual training loop
    training_steps = 100
    for step in range(training_steps):
        sess.run([train_op])
        # for debugging and learning purposes, see how the loss gets decremented thru training steps
        if step % 10 == 0:
            print ("loss: ", sess.run([loss]))

    evaluate(sess, X, Y)

import tensorflow as tf

k = tf.placeholder(tf.float32)

# Make a normal distribution, with a shifting mean
model = tf.random_normal(shape=[1000], mean=(5*k), stddev=1)
# Record that distribution into a histogram summary
tf.summary.histogram("model", model)

# Setup a session and summary writer
sess = tf.Session()
writer = tf.summary.FileWriter("irisdata")

summaries = tf.summary.merge_all()

# Setup a loop and write the summaries to disk
N = 400
for step in range(N):
  k_val = step/float(N)
  summ = sess.run(summaries, feed_dict={k: k_val})
  writer.add_summary(summ, global_step=step)
    
coord.request_stop()
coord.join(threads)
sess.close()

INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.NotFoundError'>, NewRandomAccessFile failed to Create/Open: Users/Devonirisdata : The system cannot find the path specified.
; No such process
	 [[Node: ReaderReadV2 = ReaderReadV2[_device="/job:localhost/replica:0/task:0/device:CPU:0"](TextLineReaderV2, input_producer)]]


FailedPreconditionError: Attempting to use uninitialized value Variable_33
	 [[Node: Variable_33/read = Identity[T=DT_FLOAT, _class=["loc:@Variable_33"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](Variable_33)]]

Caused by op 'Variable_33/read', defined at:
  File "C:\Users\Devon\Anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Devon\Anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Devon\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Devon\Anaconda3\envs\tensorflow\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\Devon\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "C:\Users\Devon\Anaconda3\envs\tensorflow\lib\sit